In [1]:
import requests as rq
import requests_cache
import lxml.html as lx
from bs4 import BeautifulSoup
import pandas as pd
import re
from io import StringIO

In [13]:
urls = ["https://ucr.fbi.gov/crime-in-the-u.s/2019/crime-in-the-u.s.-2019/tables/table-8/table-8-state-cuts/california.xls",
        "https://ucr.fbi.gov/crime-in-the-u.s/2018/crime-in-the-u.s.-2018/tables/table-8/table-8-state-cuts/california.xls",
        "https://ucr.fbi.gov/crime-in-the-u.s/2017/crime-in-the-u.s.-2017/tables/table-8/table-8-state-cuts/california.xls",
        "https://ucr.fbi.gov/crime-in-the-u.s/2016/crime-in-the-u.s.-2016/tables/table-6/table-6-state-cuts/california.xls",
        "https://ucr.fbi.gov/crime-in-the-u.s/2015/crime-in-the-u.s.-2015/tables/table-8/table-8-state-pieces/table_8_offenses_known_to_law_enforcement_california_by_city_2015.xls",
        "https://ucr.fbi.gov/crime-in-the-u.s/2014/crime-in-the-u.s.-2014/tables/table-8/table-8-by-state/Table_8_Offenses_Known_to_Law_Enforcement_by_California_by_City_2014.xls",
        "https://ucr.fbi.gov/crime-in-the-u.s/2013/crime-in-the-u.s.-2013/tables/table-8/table-8-state-cuts/table_8_offenses_known_to_law_enforcement_california_by_city_2013.xls",
        "https://ucr.fbi.gov/crime-in-the-u.s/2012/crime-in-the-u.s.-2012/tables/8tabledatadecpdf/table-8-state-cuts/table_8_offenses_known_to_law_enforcement_by_california_by_city_2012.xls",
        "https://ucr.fbi.gov/crime-in-the-u.s/2011/crime-in-the-u.s.-2011/tables/table8statecuts/table_8_offenses_known_to_law_enforcement_california_by_city_2011.xls",
        "https://ucr.fbi.gov/crime-in-the-u.s/2010/crime-in-the-u.s.-2010/tables/table-8/10tbl08ca.xls",]
session = requests_cache.CachedSession("crime_data")

In [14]:
request = session.get("https://www2.fbi.gov/ucr/cius2009/data/table_08_ca.html")
soup = BeautifulSoup(request.content, 'html.parser')
table = soup.find_all("table")
data_frame = pd.read_html(StringIO(str(table)))[0]
#data_frame = data_frame.drop(["Rape (legacy definition)2"], axis = 1) #2014-2016, 2013 drop "Rape (Rape(revised definition)1)
data_frame

ValueError: No tables found

In [16]:
def get_data(url_lst):
    all_data = pd.DataFrame()
    year = 2019
    names_lst = ['city', 'population', 'violent crime', 'murder and nonnegligent manslaughter', 'rape', 'robbery', 'aggravated assault',
                'property crime', 'burglary', 'larceny-theft', 'motor vehicle theft', 'arson']
    for url in url_lst:
        request = session.get(url)
        soup = BeautifulSoup(request.content, 'html.parser')
        table = soup.find("table", {"class": "data"})
        data_frame = pd.read_html(StringIO(str(table)))[0]
        if year >= 2014 and year <= 2016:
            data_frame = data_frame.drop(data_frame.columns[5], axis = 1)
        elif year == 2013:
            data_frame = data_frame.drop(data_frame.columns[4], axis = 1)
        data_frame.columns = data_frame.columns[:0].to_list() + names_lst
        state_name = "California"
        data_frame["state"] = state_name
        data_frame["year"] = year
        year -= 1
        all_data = pd.concat([all_data, data_frame])
    all_data = all_data.reset_index()
    all_data = all_data.drop(["index"], axis= 1)
    return all_data

In [17]:
california_data = get_data(urls)

In [18]:
california_data

,city,population,violent crime,murder and nonnegligent manslaughter,rape,robbery,aggravated assault,property crime,burglary,larceny-theft,motor vehicle theft,arson,state,year
0,Adelanto,34491,276,1,20.0,42,213,459.0,136.0,209.0,114,14,California,2019
1,Agoura Hills,20490,21,0,6.0,4,11,306.0,66.0,223.0,17,0,California,2019
2,Alameda,78907,162,0,7.0,94,61,2579.0,218.0,1958.0,403,29,California,2019
3,Albany,20083,40,0,8.0,21,11,685.0,105.0,534.0,46,1,California,2019
4,Alhambra,84837,161,2,11.0,89,59,1749.0,259.0,1303.0,187,8,California,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4601,Yountville,3293,5,0,1.0,1,3,57.0,9.0,47.0,1,1,California,2010
4602,Yreka,7345,54,0,2.0,2,50,263.0,69.0,176.0,18,2,California,2010
4603,Yuba City,62694,216,2,22.0,51,141,1783.0,409.0,1222.0,152,11,California,2010
4604,Yucaipa,50939,139,0,10.0,31,98,805.0,233.0,464.0,108,9,California,2010


In [39]:
crime_2009 = pd.read_excel("../raw_data/09tbl08ca.xls")
crime_2009.columns = ['city', 'population', 'violent crime', 'murder and nonnegligent manslaughter', 'rape', 'robbery', 'aggravated assault',
                'property crime', 'burglary', 'larceny-theft', 'motor vehicle theft', 'arson']
crime_2009 = crime_2009.tail(crime_2009.shape[0]-4)
crime_2009 = crime_2009.head(crime_2009.shape[0]-1)
crime_2009

,city,population,violent crime,murder and nonnegligent manslaughter,rape,robbery,aggravated assault,property crime,burglary,larceny-theft,motor vehicle theft,arson
4,Adelanto,30045,272,2,8,40,222,797,346,324,127,13
5,Agoura Hills,22469,27,0,2,6,19,308,105,192,11,6
6,Alameda,70372,199,4,13,87,95,1955,325,1390,240,12
7,Albany,15950,45,0,1,36,8,600,112,381,107,17
8,Alhambra,85956,248,0,15,133,100,1874,360,1209,305,4
...,...,...,...,...,...,...,...,...,...,...,...,...
459,Yountville,3265,4,0,2,0,2,43,11,31,1,0
460,Yreka,7379,55,0,2,4,49,224,48,162,14,0
461,Yuba City,62495,196,1,16,44,135,1555,289,1130,136,10
462,Yucaipa,50782,128,4,13,19,92,1030,247,640,143,22


In [44]:
crime_2007 = pd.read_excel("../raw_data/07tbl08ca.xls")
crime_2007 = crime_2007.drop(["Table 8"], axis = 1)
crime_2007

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,City,Population,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson1
3,Adelanto,28719,153,5,7,33,108,758,354,308,96,11
4,Agoura Hills,22966,49,0,3,9,37,348,76,254,18,6
...,...,...,...,...,...,...,...,...,...,...,...,...
459,Yucaipa,51624,52,2,9,16,25,914,250,504,160,3
460,Yucca Valley,20865,74,1,4,7,62,563,164,297,102,8
461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
